In [1]:
import requests
import pandas as pd
import numpy as np

response_API = requests.get('https://api.openalex.org')
response_API.status_code # 200 means success

200

In [2]:
# PaperAuthorAffiliations


# 333,457 affiliated works for University of Washington

# https://api.openalex.org/works?filter=institutions.id:I201448701


# last institution {} issue:
# https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszMiwgJ2h0dHBzOi8vb3BlbmFsZXgub3JnL1czMTg1MzQ4NTg4J10i

# author id issue:


UW_works_url = "https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyMjUxNTk3NjcnXSI="


UW_works = requests.get(UW_works_url).json()

rows = []
count = 0
for r in UW_works['results']:
    
    if 'id' in r:
        paper_id = r['id'].rsplit('/W', 1)[-1]
    else: 
        paper_id = np.nan

    
    for a in r['authorships']:
        count += 1
        if 'id' in a['author']:
            author_id = a['author']['id'].rsplit('/A', 1)[-1]
        else:
            author_id = np.nan
            
        if 'display_name' in a['author']:
            author_display_name = a['author']['display_name']
        else:
            author_display_name = np.nan
        
        if 'author_position' in a:
            author_position = a['author_position']
        else:
            author_position = np.nan

        
        # get the second last instition if last_instition is an empty dictionary {}        
        institutions_list = a['institutions']
        
        if len(institutions_list) != 0 and isinstance(institutions_list[-1], dict) and not bool(institutions_list[-1]):
            institutions_list.pop()
                    
        if len(institutions_list) != 0:
            last_instition = institutions_list[-1:][0]
            
            if 'id' in last_instition and last_instition['id'] != None:
                aff_id = last_instition['id'].rsplit('/I', 1)[-1]
            else:
                aff_id = np.nan
            
            if 'display_name' in last_instition:
                aff_display_name = last_instition['display_name']
            else:
                aff_display_name = np.nan
            
            if 'country_code' in last_instition:
                country_code = last_instition['country_code']
            else:
                country_code = np.nan
            
        else:
            aff_id = np.nan
            display_name = np.nan
            contry_code = np.nan
    

        rows.append([paper_id, author_id, aff_id, aff_display_name, country_code, 
                     author_position, author_display_name])
    
df = pd.DataFrame(rows, columns=["Paper_id", "Author_ID", "Affiliation_ID", 
                                          "Affiliation_name", "Country_code", 
                                          "Author_sequence_number", "Author_name"])
df

,Paper_id,Author_ID,Affiliation_ID,Affiliation_name,Country_code,Author_sequence_number,Author_name
0,4225286463,2952135929,114027177,University of North Carolina at Chapel Hill,US,first,Le Huang
1,4225286463,2587329117,114027177,University of North Carolina at Chapel Hill,US,middle,Jonathan D. Rosen
2,4225286463,2279220919,114027177,University of North Carolina at Chapel Hill,US,middle,Quan Sun
3,4225286463,2111358758,114027177,University of North Carolina at Chapel Hill,US,middle,Jiawen Chen
4,4225286463,2149429328,201448701,University of Washington,US,middle,Marsha M. Wheeler
...,...,...,...,...,...,...,...
333,4237326040,1986295364,NaN,"Department of Materials Science & Engineering,...",None,middle,Stephanie J. Benight
334,4237326040,629959257,NaN,"Department of Materials Science & Engineering,...",None,middle,Ilya Kosilkin
335,4237326040,2162628756,NaN,"Department of Materials Science & Engineering,...",None,middle,Bruce G. Robinson
336,4237326040,2136157913,NaN,"Department of Materials Science & Engineering,...",None,middle,Bruce E. Eichinger


In [3]:
UW_works_url_with_cursor = 'https://api.openalex.org/works?filter=institutions.id:I201448701&cursor={}'


cursor = '*'
PaperAuthorAffiliations_rows = []

count = 1

# loop through pages
while cursor:
    UW_works_url = UW_works_url_with_cursor.format(cursor)
    print("\n" + str(count) + ": " + UW_works_url)

    UW_works = requests.get(UW_works_url).json()
        
    for r in UW_works['results']:
        if 'id' in r:
            paper_id = r['id'].rsplit('/W', 1)[-1]
        else: 
            paper_id = np.nan


        for a in r['authorships']:
            if 'id' in a['author']:
                author_id = a['author']['id'].rsplit('/A', 1)[-1]
            else:
                author_id = np.nan

            if 'display_name' in a['author']:
                author_display_name = a['author']['display_name']
            else:
                author_display_name = np.nan

            if 'author_position' in a:
                author_position = a['author_position']
            else:
                author_position = np.nan


            # get the second last instition if last_instition is an empty dictionary {}        
            institutions_list = a['institutions']
            if len(institutions_list) != 0 and isinstance(institutions_list[-1], dict) and not bool(institutions_list[-1]):
                institutions_list.pop()

            if len(institutions_list) != 0:
                last_instition = institutions_list[-1:][0]

                if 'id' in last_instition and last_instition['id'] != None:
                    aff_id = last_instition['id'].rsplit('/I', 1)[-1]
                else:
                    aff_id = np.nan

                if 'display_name' in last_instition:
                    aff_display_name = last_instition['display_name']
                else:
                    aff_display_name = np.nan

                if 'country_code' in last_instition:
                    country_code = last_instition['country_code']
                else:
                    country_code = np.nan

            else:
                aff_id = np.nan
                display_name = np.nan
                contry_code = np.nan


            PaperAuthorAffiliations_rows.append([paper_id, author_id, aff_id, 
                                                 aff_display_name, country_code, 
                                                 author_position, author_display_name])

    # update cursor to meta.next_cursor
    cursor = UW_works['meta']['next_cursor']
    
    count += 1


PaperAuthorAffiliations_df = pd.DataFrame(PaperAuthorAffiliations_rows, 
                                          columns=["Paper_id", "Author_ID", "Affiliation_ID", 
                                          "Affiliation_name", "Country_code", 
                                          "Author_sequence_number", "Author_name"])
PaperAuthorAffiliations_df


1: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=*

2: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3ODQyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIxNTc5MTc0MTEnXSI=

3: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1Mjc5LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwOTYxMTMyMzYnXSI=

4: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0NDMyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwNjUzMDQzNTMnXSI=

5: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MDI2LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIxNzQ2NjE3NDknXSI=

6: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDE0LCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIwMTE4MzI5NjInXSI=

7: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszMTkyLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzIxNjQwNzU2NDYnXSI=

8: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlsyODE4LCAnaHR0cHM6Ly9v


61: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils4MjEsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjEzNDc0ODExNCddIg==

62: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils4MTEsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjA4ODk5MTg0OCddIg==

63: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils4MDUsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAwMDk4NTE1OCddIg==

64: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3OTcsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE0ODgxNzUzNyddIg==

65: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3OTAsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjA1OTU3ODk0OSddIg==

66: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3ODIsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAyNDg0MzAwNiddIg==

67: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils3NzcsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjc4ODA0NDExMyddIg==

68: https://api.openalex.org/work


121: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MzgsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAwNDEwMTAwMiddIg==

122: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MzUsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjA0NzIxNDAwNiddIg==

123: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MzIsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAyMTA4NTQ2MiddIg==

124: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MjksICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjA1MDgzODI3NiddIg==

125: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MjYsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjEyMjQ1NTU1NyddIg==

126: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MjMsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAzMzEzMjg5MCddIg==

127: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils1MjAsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAwODcwNjY1OSddIg==

128: https://api.openalex.


180: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTksICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjA4MzY0NTcxMCddIg==

181: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTgsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE3MzQ5NTg0MiddIg==

182: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTYsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMTYwODU2MzI2MSddIg==

183: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTYsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE2Mjg3MTAwNyddIg==

184: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTUsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE1NTI4NTAxOSddIg==

185: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTMsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMTk3Mjk2ODY3NiddIg==

186: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=Ils0MTMsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMzA5MTIxNDAyMCddIg==

187: https://api.openalex.


239: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDcsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjAzODgzMzY4OCddIg==

240: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDYsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMTc2MTAyMjEzOSddIg==

241: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDYsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjkxMDUwODQxNCddIg==

242: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDUsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE0NTI4NzE0OSddIg==

243: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDQsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE2ODkwOTA3NSddIg==

244: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDMsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjE0MDIwMjQ5MyddIg==

245: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlszNDIsICdodHRwczovL29wZW5hbGV4Lm9yZy9XMjA5NTcwNDM0NCddIg==

246: https://api.openalex.

...


13086: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTcwMDM5NTInXSI=

13087: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTcyMjQ0OTQnXSI=

13088: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTczNDYzMjInXSI=

13089: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTc0OTc0MDInXSI=

13090: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTc4MjIxNDInXSI=

13091: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTc4OTg3ODYnXSI=

13092: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQyOTgwMDM4MDAnXSI=

13093: https://api.openalex.org/works?fi


13146: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMDk2NjQ2ODEnXSI=

13147: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMDk4NDA2MjInXSI=

13148: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTAwMTAzOTUnXSI=

13149: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTAwOTYxODInXSI=

13150: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTAxMDcxMDInXSI=

13151: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTAxMjQ0OTcnXSI=

13152: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTAxNzQxNjgnXSI=

13153: https://api.openalex.org/works?fi


13206: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTM1OTQ3NTInXSI=

13207: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTM2NDcyODInXSI=

13208: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTM2NTU1NzQnXSI=

13209: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTM4Mjc4MzYnXSI=

13210: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTM4OTgzMzEnXSI=

13211: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTU0ODk4ODQnXSI=

13212: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMTU2NDM1NTEnXSI=

13213: https://api.openalex.org/works?fi


13266: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMjI0OTIzOTYnXSI=

13267: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzMjI1ODAzMTQnXSI=

13268: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQzODI5NzgwJ10i

13269: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzQ3OTA4NzI4MSddIg==

13270: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzUxMTI5NzMwJ10i

13271: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzU0NjA3NDQnXSI=

13272: https://api.openalex.org/works?filter=institutions.id:I201448701&cursor=IlswLCAnaHR0cHM6Ly9vcGVuYWxleC5vcmcvVzU2ODQyODE5J10i

13273: https://api.openalex.org/works?filter=institution

,Paper_id,Author_ID,Affiliation_ID,Affiliation_name,Country_code,Author_sequence_number,Author_name
0,2030966943,354062768,201448701,University of Washington,US,first,Joseph Felsenstein
1,2142225512,2642964564,64045040,Fooyin University,TW,first,Hsiu Ching Laura Hsieh
2,2142225512,2111315299,201448701,University of Washington,US,last,Sarah E. Shannon
3,1980216394,2104839341,5124864,École Polytechnique Fédérale de Lausanne,CH,first,Brian C. O’Regan
4,1980216394,2012626572,5124864,École Polytechnique Fédérale de Lausanne,CH,last,Michael Grätzel
...,...,...,...,...,...,...,...
1901578,99980466,2180837181,4210131277,Instituto de Microcirugía Ocular,ES,middle,José L. Güell
1901579,99980466,2111176233,4210131277,Instituto de Microcirugía Ocular,ES,middle,Merce Morral
1901580,99980466,2077428892,4210131277,Instituto de Microcirugía Ocular,ES,middle,Oscar Gris
1901581,99980466,2127986247,4210131277,Instituto de Microcirugía Ocular,ES,middle,Javier Gaytan


In [4]:
PaperAuthorAffiliations_df.to_csv(r'/Users/zhiyuwang/Desktop/Rosetta/openAlex/csv/paperAuthorAffiliations.csv', index=False)
